## Design a three-layer ANN classifier

# Set up environment

In [38]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense

#Load and inspect data

In [39]:
churn = pd.read_csv('Churn_Modelling.csv')

# Display basic information
print(churn.shape)
print('\n')
print(churn.describe())
print('\n')
print(churn.info())

(10000, 14)


         RowNumber    CustomerId   CreditScore           Age        Tenure  \
count  10000.00000  1.000000e+04  10000.000000  10000.000000  10000.000000   
mean    5000.50000  1.569094e+07    650.528800     38.921800      5.012800   
std     2886.89568  7.193619e+04     96.653299     10.487806      2.892174   
min        1.00000  1.556570e+07    350.000000     18.000000      0.000000   
25%     2500.75000  1.562853e+07    584.000000     32.000000      3.000000   
50%     5000.50000  1.569074e+07    652.000000     37.000000      5.000000   
75%     7500.25000  1.575323e+07    718.000000     44.000000      7.000000   
max    10000.00000  1.581569e+07    850.000000     92.000000     10.000000   

             Balance  NumOfProducts    HasCrCard  IsActiveMember  \
count   10000.000000   10000.000000  10000.00000    10000.000000   
mean    76485.889288       1.530200      0.70550        0.515100   
std     62397.405202       0.581654      0.45584        0.499797   
min        

#Clean data

In [40]:
# Drop identifying columns
churn = churn.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

# Encode 'Gender' (binary category)
churn['Gender'] = LabelEncoder().fit_transform(churn['Gender'])

# One-hot encode 'Geography' (multi-category)
churn = pd.get_dummies(churn, columns=['Geography'], drop_first=True)

print(churn.describe())

        CreditScore        Gender           Age        Tenure        Balance  \
count  10000.000000  10000.000000  10000.000000  10000.000000   10000.000000   
mean     650.528800      0.545700     38.921800      5.012800   76485.889288   
std       96.653299      0.497932     10.487806      2.892174   62397.405202   
min      350.000000      0.000000     18.000000      0.000000       0.000000   
25%      584.000000      0.000000     32.000000      3.000000       0.000000   
50%      652.000000      1.000000     37.000000      5.000000   97198.540000   
75%      718.000000      1.000000     44.000000      7.000000  127644.240000   
max      850.000000      1.000000     92.000000     10.000000  250898.090000   

       NumOfProducts    HasCrCard  IsActiveMember  EstimatedSalary  \
count   10000.000000  10000.00000    10000.000000     10000.000000   
mean        1.530200      0.70550        0.515100    100090.239881   
std         0.581654      0.45584        0.499797     57510.492818   

#Split into Train(80%) and Test(20%)

In [41]:
# Split features and target
X = churn.drop('Exited', axis=1)
y = churn['Exited']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
              X, y, test_size=0.2, random_state=680)

#Feature Scaling

In [42]:
# Standardize features
scaler = StandardScaler()
X_train_scale = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#Build ANN model

In [45]:
# Build the model
model = keras.Sequential([
    keras.Input(shape=(X_train_scale.shape[1],)),
    layers.Dense(units=16, activation='relu'),
    layers.Dense(units=16, activation='relu'),
    layers.Dense(units=1, activation='sigmoid')])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(X_train_scale, y_train, epochs=50, batch_size=32, verbose=0)

#Evaluate model

In [46]:
# Evaluate the model
y_pred = (model.predict(X_test_scaled) > 0.5)

# Display
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Accuracy: 0.8535
Confusion Matrix:
 [[1519   80]
 [ 213  188]]


#END